In [1]:
import pandas as pd
testdata = pd.read_csv("test.csv")
traindata = pd.read_csv("train.csv")

In [2]:
data = pd.concat([traindata,testdata])
data = data.sample(frac=0.02).reset_index(drop=True)

In [3]:
data.drop(labels=["dataset"], axis=1)

,text,label
0,Ancak buna rağmen Irak’ın işgaline karşı durmu...,Notr
1,"uzun süredir bu ürünü kullanıyorum, kullanmaya...",Positive
2,çok güzel bir tatil geçirdik otel patronları o...,Positive
3,ürünü cuma günü sipariş ettim pazartesi elimde...,Positive
4,Köy statüsünde olmasına rağmen Karabük şehrine...,Notr
...,...,...
9788,Tefal ismine güvenerek aldım.. ilk çalıştırma ...,Negative
9789,Kural kısa sürede internet fenomeni haline gel...,Notr
9790,"adının hakkını veriyor, gerçekten mükemmel",Positive
9791,Bu albümü kendi hayranlarından oluşturduğu dağ...,Notr


In [4]:
data["label"].value_counts()

label
Positive    5241
Notr        3409
Negative    1143
Name: count, dtype: int64

In [5]:
import string
import re
import nltk
from nltk.corpus import stopwords

noktalama = string.punctuation
etkisiz = stopwords.words('turkish')

In [6]:
def etkisizTemizleme(metin):
    temp = ''
    for word in metin.split():
        if word not in etkisiz and not word.isnumeric():
            temp += word + ' '
    #print(temp + "\n***************************")
    return temp
    

def noktaTemizleme(metin):
    temp = ''
    for word in metin:
        if word not in noktalama:
            temp += word
    #print(temp + "\n***************************")
    d = temp
    return d

In [7]:
data["text2"] = data["text"].apply(etkisizTemizleme)

In [8]:
data["text3"] = data["text2"].apply(noktaTemizleme)

In [9]:
data.head()

,text,label,dataset,text2,text3
0,Ancak buna rağmen Irak’ın işgaline karşı durmu...,Notr,wiki,Ancak buna rağmen Irak’ın işgaline karşı durmu...,Ancak buna rağmen Irak’ın işgaline karşı durmu...
1,"uzun süredir bu ürünü kullanıyorum, kullanmaya...",Positive,urun_yorumlari,"uzun süredir ürünü kullanıyorum, kullanmaya ba...",uzun süredir ürünü kullanıyorum kullanmaya baş...
2,çok güzel bir tatil geçirdik otel patronları o...,Positive,HUMIR,güzel bir tatil geçirdik otel patronları olsun...,güzel bir tatil geçirdik otel patronları olsun...
3,ürünü cuma günü sipariş ettim pazartesi elimde...,Positive,urun_yorumlari,ürünü cuma günü sipariş ettim pazartesi elimde...,ürünü cuma günü sipariş ettim pazartesi elimde...
4,Köy statüsünde olmasına rağmen Karabük şehrine...,Notr,wiki,Köy statüsünde olmasına rağmen Karabük şehrine...,Köy statüsünde olmasına rağmen Karabük şehrine...


In [10]:
newData = data.drop(["text2","dataset","text"],axis=1)

In [11]:
newData.head()

,label,text3
0,Notr,Ancak buna rağmen Irak’ın işgaline karşı durmu...
1,Positive,uzun süredir ürünü kullanıyorum kullanmaya baş...
2,Positive,güzel bir tatil geçirdik otel patronları olsun...
3,Positive,ürünü cuma günü sipariş ettim pazartesi elimde...
4,Notr,Köy statüsünde olmasına rağmen Karabük şehrine...


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newData["text3"].values,newData["label"],test_size=0.1)

In [13]:
X_train.shape

(8813,)

In [14]:
X_test.shape

(980,)

In [15]:
#Sayma vektörlerini oluşturuyoruz
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(8813, 38296)

In [16]:
#Sayma vektörlerini tf_idf vektörlerine çeviriyoruz
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(8813, 38296)

In [17]:
#çok modlu naive bayes sınıflandırıcı eğitiyoruz
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [18]:
#Sınıflandırıcı ile test seti üzerinde tahminleme yapıyoruz
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:5],y_pred[:]):
    print('%r => %s' % (review, sentiment))

'Bu harita yeni oyuncuların oyunun amacını anlaması içindir  ' => Notr
'günde teslim edildikullanışlı kapı pervazında vidalamaya gerek olmadan takılı şekilde kullanılabiliyor ' => Positive
'anlamıyorsanız tür işlerden bulaşmayın derim tam bir başa bela ' => Positive
'öncelikle hb teşekkür ederiz verdiği himetten dolayi ürünü yaklaşik yıldır kullanıyorum memnunum hiçbir sorunum olmadı kargo hizmetinde mükemmel tavsiye ederim şimdi kayınvalideme alıyorum sorunsuz bir ürün ' => Positive
'1 makinenin çekişi kuvvetli500w ' => Positive


In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8030612244897959


In [37]:
test_metni = ["Bu nasıl bir cümle?"]
test_count = count_vect.transform(test_metni)
test_tfidf = tfidf_transformer.transform(test_count)

test_tahmin = clf.predict(test_tfidf)

In [38]:
test_tahmin

array(['Positive'], dtype='<U8')